In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import re
from nltk.corpus import stopwords

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [2]:
train = pd.read_csv("../csv/train.csv")

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
# 数を確認
print(len(train))

159571


In [5]:
# idを左側の軸にする
train.index = train['id']
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,,
0000997932d777bf,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
000103f0d9cfb60f,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
000113f07ec002fd,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
0001b41b1c6bb37e,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
0001d958c54c6e35,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
x_train = train['comment_text']
x_train.head()

id
0000997932d777bf    Explanation\nWhy the edits made under my usern...
000103f0d9cfb60f    D'aww! He matches this background colour I'm s...
000113f07ec002fd    Hey man, I'm really not trying to edit war. It...
0001b41b1c6bb37e    "\nMore\nI can't make any real suggestions on ...
0001d958c54c6e35    You, sir, are my hero. Any chance you remember...
Name: comment_text, dtype: object

In [7]:
y_train = train.iloc[:, 2:]
y_train.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
0000997932d777bf,0,0,0,0,0,0
000103f0d9cfb60f,0,0,0,0,0,0
000113f07ec002fd,0,0,0,0,0,0
0001b41b1c6bb37e,0,0,0,0,0,0
0001d958c54c6e35,0,0,0,0,0,0


In [8]:
# cleanカラムを作成する

# 列方向に足し合わせる
# 1以下なのをFalseとする
y_train["clean"] = 1 - y_train.sum(axis=1) >= 1
y_train.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
id,,,,,,,
0000997932d777bf,0,0,0,0,0,0,True
000103f0d9cfb60f,0,0,0,0,0,0,True
000113f07ec002fd,0,0,0,0,0,0,True
0001b41b1c6bb37e,0,0,0,0,0,0,True
0001d958c54c6e35,0,0,0,0,0,0,True


In [9]:
kinds, counts = zip(*y_train.sum(axis=0).items())
kinds

('toxic',
 'severe_toxic',
 'obscene',
 'threat',
 'insult',
 'identity_hate',
 'clean')

In [10]:
counts

(15294.0, 1595.0, 8449.0, 478.0, 7877.0, 1405.0, 143346.0)

In [11]:
bars = go.Bar(
         y=counts,
         x=kinds,
       )

layout = go.Layout(title="Class distribution in train set")
fig = go.Figure(data=[bars], layout=layout)
iplot(fig, filename="bar")

In [12]:
y_train.sum(axis=0).items()

In [13]:
for kind in y_train.columns:
    # y_train[kind] == 1 → cleanが1なのを3つ持ってくる
    x_kind = x_train[y_train[kind] == 1].sample(3)
    # print("\n".join(x_kind))
    print(x_kind)

id
2ed5af1e888a6295    this guy callled Kansas the arm taker is nothi...
937c25263bf86e70    And go ahead and delete my account, I can't wa...
0325d248b8875710    "\n\nSo threatening to try to have me banned f...
Name: comment_text, dtype: object
id
ee32164a4d102aa7    "\n\nFuck off you stupid fucking retard cunt.\...
741425433bba9e36    You know what. Fuck you! I wan'teven done writ...
8c3758253064f2e6    Feydey is a complete fucking fag. I saw that h...
Name: comment_text, dtype: object
id
33246ffcc0a9f8d9    Dear Sinebot\n\nFuck you.  In case you didn't ...
76064f0b387120bb    Wash your dirty penis before giving shit \n\nW...
7f6c52d763badf29    Ok, buddy! im rally getting pissed. i was goin...
Name: comment_text, dtype: object
id
69123690b8d730c4    You leave my friend's editing alone! \n\nHello...
188ec3b7baf66495    Urgent!!!! \n\nMind yo own fucken buisiness st...
e6a551f670c58ded    BTW, I love to harrass, so try banning me for ...
Name: comment_text, dtype: object
id
60f3ee86b

## テキスト標準化

### spacy 
https://github.com/explosion/spaCy

### spacyエラー対応
https://teratail.com/questions/173701

In [14]:
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
stops = stopwords.words("english")

In [15]:
def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    
    return " ".join(lemmatized)

In [16]:
x_train_lemmatized = x_train.apply(normalize, lowercase=True, remove_stopwords=True)

In [17]:
# 一つデータを確認
x_train_lemmatized.sample(1).iloc[0]

'good well reason include ! page empty edit yesterday - along site relate ips add . guess glitch site since show content . understand point numb 2 . claim reference - wikilink provide guideline external link , site fall foul newish open wiki without substantial edit history many contributor , particular reputation subject . may great subject , mean expand issue create big association untrustworthy page . remove link page - may good right . try push pov remove one . clean campaign spam wikipedia link particular website look link whole . would encourage clean link credible , putt pov comment site appropriate context , -PRON- sure would know begin subject like . siobhanhansa'

## 単語頻度の視覚化

In [18]:
from collections import Counter

word_counts = dict()

for kind in y_train.columns:
    word_counts[kind] = Counter()
    comments = x_train_lemmatized[y_train[kind]==1]
    for _, comment in comments.iteritems():
        word_counts[kind].update(comment.split(" "))

In [19]:
def most_common_words(kind, num_words=15):
    words, counts = zip(*word_counts[kind].most_common(num_words)[::-1])
    bars = go.Bar(
        y=words,
        x=counts,
        orientation="h"
    )

    layout = go.Layout(
        title="Most common words of the class \"{}\"".format(kind),
        yaxis=dict(
            ticklen=8  # to add some space between yaxis labels and the plot
        )
    )

    fig = go.Figure(data=[bars], layout=layout)
    iplot(fig, filename='bar')

In [20]:
most_common_words("toxic", 30)

In [21]:
most_common_words("severe_toxic")

### Counter使ってみる
https://qiita.com/hatchinee/items/a904c1f8d732a4686c9d

In [22]:
data = ["aaa", "bbb", "ccc", "aaa"]
data

['aaa', 'bbb', 'ccc', 'aaa']

In [23]:
counter = Counter(data)
counter

Counter({'aaa': 2, 'bbb': 1, 'ccc': 1})

In [24]:
for word, cnt in counter.most_common():
    print(word, cnt)

aaa 2
bbb 1
ccc 1
